In [1]:
# 코드를 실행시키기 위한 작업을 실행합니다.
# 시작할 때 반드시 T4-GPU 설정으로 돌리세요!
!pip install timm albumentations

In [2]:
# [Cell 1] Kaggle 데이터셋 다운로드 및 환경 설정
import os
from google.colab import files

# 1. kaggle.json 업로드
if not os.path.exists('kaggle.json'):
    print("🔻 kaggle.json 파일을 업로드해주세요.")
    files.upload()

# 2. Kaggle 인증 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 3. 데이터 다운로드 (140k-real-and-fake-faces)
# 이미 얼굴이 잘려(Crop) 있어서 전처리 없이 바로 학습 가능
if not os.path.exists('./data'):
    print("🔻 데이터 다운로드 중... (약 2~3분 소요)")
    !kaggle datasets download -d xhlulu/140k-real-and-fake-faces
    !unzip -q 140k-real-and-fake-faces.zip -d ./data
    print("✅ 데이터 준비 완료! (./data 폴더 확인)")
else:
    print("✅ 이미 데이터가 존재합니다.")

🔻 kaggle.json 파일을 업로드해주세요.


Saving kaggle.json to kaggle.json
🔻 데이터 다운로드 중... (약 2~3분 소요)
Dataset URL: https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces
License(s): other
 99% 3.73G/3.75G [00:37<00:00, 297MB/s]
100% 3.75G/3.75G [00:37<00:00, 107MB/s]
✅ 데이터 준비 완료! (./data 폴더 확인)


In [3]:
# [Cell 2] 프로젝트 폴더 구조 생성
import os

folders = ['model', 'src', 'config', 'train_data', 'test_data']
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("✅ 폴더 구조 생성 완료: model/, src/, config/ 등")

✅ 폴더 구조 생성 완료: model/, src/, config/ 등


In [4]:
%%writefile config/config.yaml
# [Config] 실험 설정 파일
SEED: 42
IMG_SIZE: 224        # EfficientNet-B4라면 380 권장, 빠른 실험 위해 224 설정
BATCH_SIZE: 32
EPOCHS: 10
LEARNING_RATE: 0.0001
MODEL_NAME: 'tf_efficientnet_b4_ns' # 추천 모델
DATA_PATH: './data/real_vs_fake/real-vs-fake' # Kaggle 데이터 경로
SAVE_PATH: './model/best_model.pt'

Writing config/config.yaml


In [5]:
%%writefile src/dataset.py
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

class DeepfakeDataset(Dataset):
    def __init__(self, file_paths, labels=None, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        image = cv2.imread(path)

        # 예외처리: 이미지가 안 읽힐 경우 검은 화면 반환
        if image is None:
            image = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        if self.labels is not None:
            return image, torch.tensor(self.labels[idx], dtype=torch.float32)
        else:
            return image

def get_transforms(img_size):
    # 학습용 증강
    train_aug = A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        A.GaussNoise(p=0.2), # 노이즈 추가 (화질구지 대응)
        A.Normalize(),
        ToTensorV2()
    ])
    # 검증/추론용 (증강 없음)
    valid_aug = A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(),
        ToTensorV2()
    ])
    return train_aug, valid_aug

Writing src/dataset.py


In [6]:
%%writefile src/models.py
import torch.nn as nn
import timm

class DeepfakeModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(DeepfakeModel, self).__init__()
        # 1. 모델 로드 (pretrained=False면 껍데기만 로드됨)
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=1)

    def forward(self, x):
        return self.model(x)

Writing src/models.py


In [7]:
%%writefile train.py
import os
import glob
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# 위에서 만든 모듈 불러오기
from src.dataset import DeepfakeDataset, get_transforms
from src.models import DeepfakeModel

# Config 로드
with open('config/config.yaml') as f:
    CFG = yaml.safe_load(f)

def train():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using Device: {device}")

    # 1. 데이터 로드 (Kaggle 데이터 구조에 맞춤)
    print("Load Data...")
    base_path = CFG['DATA_PATH']

    # Real: 0, Fake: 1
    real_paths = glob.glob(os.path.join(base_path, 'train/real/*.jpg'))
    fake_paths = glob.glob(os.path.join(base_path, 'train/fake/*.jpg'))

    # 빠른 테스트를 위해 데이터 일부만 사용 (필요시 주석 해제하여 전체 사용)
    # real_paths = real_paths[:1000]
    # fake_paths = fake_paths[:1000]

    all_paths = real_paths + fake_paths
    labels = [0]*len(real_paths) + [1]*len(fake_paths)

    # Train/Valid Split
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        all_paths, labels, test_size=0.2, random_state=CFG['SEED'], stratify=labels
    )

    # Transforms
    train_tf, valid_tf = get_transforms(CFG['IMG_SIZE'])

    # DataLoader
    train_loader = DataLoader(
        DeepfakeDataset(train_paths, train_labels, train_tf),
        batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=2
    )
    val_loader = DataLoader(
        DeepfakeDataset(val_paths, val_labels, valid_tf),
        batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=2
    )

    # 2. 모델 준비
    model = DeepfakeModel(CFG['MODEL_NAME'], pretrained=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
    criterion = nn.BCEWithLogitsLoss()

    best_loss = float('inf')

    # 3. 학습 루프
    print("Start Training...")
    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = 0

        for imgs, targets in tqdm(train_loader, desc=f"Epoch {epoch} Train"):
            imgs, targets = imgs.to(device), targets.to(device).unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # 검증
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for imgs, targets in val_loader:
                imgs, targets = imgs.to(device), targets.to(device).unsqueeze(1)
                outputs = model(imgs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # 모델 저장 (Loss 기준)
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), CFG['SAVE_PATH'])
            print(f"✅ Best Model Saved! ({val_loss:.4f})")

if __name__ == '__main__':
    train()

Writing train.py


In [8]:
!python train.py

Using Device: cuda
Load Data...
/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(
model.safetensors: 100% 77.9M/77.9M [00:01<00:00, 60.9MB/s]
Start Training...
Epoch 1 Train: 100% 2500/2500 [19:04<00:00,  2.19it/s]
Epoch 1 | Train Loss: 0.2768 | Val Loss: 0.0401
✅ Best Model Saved! (0.0401)
Epoch 2 Train: 100% 2500/2500 [19:13<00:00,  2.17it/s]
Epoch 2 | Train Loss: 0.0961 | Val Loss: 0.0263
✅ Best Model Saved! (0.0263)
Epoch 3 Train: 100% 2500/2500 [19:12<00:00,  2.17it/s]
Epoch 3 | Train Loss: 0.0732 | Val Loss: 0.0112
✅ Best Model Saved! (0.0112)
Epoch 4 Train:   0% 7/2500 [00:04<23:54,  1.74it/s]
Traceback (most recent call last):
  File "/content/train.py", line 101, in <module>
    train()
  File "/content/train.py", line 77, in train
    train_loss += loss.item()
                  ^^^^^^^^^^^
KeyboardInterrupt


In [16]:
# [Cell] 구글 드라이브 마운트 및 지정된 6개 확장자 파일 추출
import os
import shutil
import glob
from google.colab import drive

# 1. 기존 폴더 정리 (깨끗하게 초기화)
if os.path.exists('./test_data'):
    shutil.rmtree('./test_data')
if os.path.exists('./temp_test'):
    shutil.rmtree('./temp_test')

# 2. 구글 드라이브 마운트
drive.mount('/content/drive')

# 3. 경로 설정 (사용자님 드라이브 경로)
drive_path = '/content/drive/MyDrive/딥페이크 AI 대회/test.zip'

if os.path.exists(drive_path):
    print(f"✅ 구글 드라이브에서 파일을 찾았습니다! ({os.path.getsize(drive_path)/1024/1024:.2f} MB)")

    # 압축 풀기
    print("📦 압축 해제 중... (잠시만 기다려주세요)")
    !unzip -q "{drive_path}" -d ./temp_test

    # 4. 파일 이동 (지정해주신 6개 확장자만 타겟팅)
    os.makedirs('./test_data', exist_ok=True)

    # 검색할 확장자 리스트 (대소문자 모두 포함하도록 설정)
    target_extensions = ['jfif', 'jpeg', 'jpg', 'mov', 'mp4', 'png']
    search_patterns = []

    # 예: *.jpg 와 *.JPG 를 모두 검색 리스트에 추가
    for ext in target_extensions:
        search_patterns.append(f'*.{ext.lower()}')
        search_patterns.append(f'*.{ext.upper()}')

    print(f"검색할 확장자들: {search_patterns}")

    all_files = []
    for pattern in search_patterns:
        # 재귀적으로 폴더 깊숙한 곳까지 검색
        found = glob.glob(f'./temp_test/**/{pattern}', recursive=True)
        all_files.extend(found)

    print(f"총 {len(all_files)}개의 파일을 발견했습니다. 이동을 시작합니다...")

    # 파일 이동 실행
    for file in all_files:
        try:
            # 파일명이 겹칠 경우를 대비해 덮어쓰기 혹은 건너뛰기
            shutil.move(file, './test_data')
        except shutil.Error:
            print(f"⚠️ 중복 파일 존재: {os.path.basename(file)} (건너뜀)")
        except Exception as e:
            print(f"❌ 이동 실패: {file} - {e}")

    # 최종 확인
    final_count = len(os.listdir('./test_data'))
    print(f"🎉 성공! ./test_data 폴더 안에 총 {final_count}개의 파일이 준비되었습니다.")

    # 임시 폴더 삭제
    shutil.rmtree('./temp_test')

else:
    print(f"❌ '{drive_path}' 경로에 파일이 없습니다.")
    print("구글 드라이브 경로와 파일명을 다시 한번 확인해주세요.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 구글 드라이브에서 파일을 찾았습니다! (1808.57 MB)
📦 압축 해제 중... (잠시만 기다려주세요)
검색할 확장자들: ['*.jfif', '*.JFIF', '*.jpeg', '*.JPEG', '*.jpg', '*.JPG', '*.mov', '*.MOV', '*.mp4', '*.MP4', '*.png', '*.PNG']
총 500개의 파일을 발견했습니다. 이동을 시작합니다...
🎉 성공! ./test_data 폴더 안에 총 500개의 파일이 준비되었습니다.


In [22]:
# 얼굴만 오려내는 가위를 추가하기 위해 추가적인 pip를 설치합니다.
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124

In [3]:
%%writefile inference.py
import os
import cv2
import yaml
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from facenet_pytorch import MTCNN
from src.dataset import DeepfakeDataset, get_transforms
from src.models import DeepfakeModel

def inference(test_dir, model_path, output_path):
    # --- 설정 ---
    with open('config/config.yaml') as f:
        CFG = yaml.safe_load(f)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # --- 모델 로드 ---
    model = DeepfakeModel(CFG['MODEL_NAME'], pretrained=False)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # --- 얼굴 탐지기 설정 (Kaggle 데이터처럼 타이트하게) ---
    # margin=0 : 얼굴을 타이트하게 자름
    # select_largest=True : 화면에서 제일 큰 얼굴 하나만 가져옴
    mtcnn = MTCNN(keep_all=False, select_largest=True, device=device, margin=0)

    # Transforms
    _, valid_tf = get_transforms(CFG['IMG_SIZE'])

    # --- 파일 목록 ---
    VALID_EXTENSIONS = ('.mp4', '.mov', '.avi', '.jpg', '.jpeg', '.png', '.jfif')
    file_paths = []
    for f in os.listdir(test_dir):
        if f.lower().endswith(VALID_EXTENSIONS):
            file_paths.append(os.path.join(test_dir, f))

    print(f"✅ Inference Start! 파일: {len(file_paths)}개")

    # [디버깅용] AI가 보는 이미지를 저장할 폴더 생성
    os.makedirs('./debug_images', exist_ok=True)
    saved_debug_count = 0

    results = []

    for path in tqdm(file_paths):
        filename = os.path.basename(path)
        ext = filename.split('.')[-1].lower()

        frames = []

        # --- 소스 읽기 ---
        if ext in ['mp4', 'mov', 'avi']:
            cap = cv2.VideoCapture(path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            # 영상은 중앙 프레임 위주로 5장만 뽑아서 빠르게 (속도 개선)
            if frame_count > 10:
                intervals = np.linspace(frame_count//4, frame_count*3//4, 5, dtype=int)
            else:
                intervals = [0]

            raw_frames = []
            for idx in intervals:
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                if ret:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    raw_frames.append(frame)
            cap.release()
        else:
            image = cv2.imread(path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                raw_frames = [image]
            else:
                raw_frames = []

        # --- 얼굴 추출 및 전처리 ---
        for i, frame in enumerate(raw_frames):
            # 1. 얼굴 탐지
            boxes, _ = mtcnn.detect(frame)

            face_img = None
            if boxes is not None:
                # 얼굴 찾음 -> 자르기
                x1, y1, x2, y2 = [int(b) for b in boxes[0]]
                # 좌표 보정
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)

                # 얼굴 영역 크기가 너무 작으면 무시 (노이즈일 수 있음)
                if (x2-x1) > 30 and (y2-y1) > 30:
                    face_img = frame[y1:y2, x1:x2]

            # 얼굴 못 찾았으면? -> 원본 그대로 사용 (보험)
            if face_img is None:
                face_img = frame

            # 2. Augmentation (Resize & Normalize)
            augmented = valid_tf(image=face_img)
            input_tensor = augmented['image']
            frames.append(input_tensor)

            # [디버깅] 처음 10개 이미지만 저장해서 눈으로 확인
            if saved_debug_count < 10:
                # 정규화된 텐서를 다시 이미지로 복구해서 저장
                debug_img = input_tensor.permute(1, 2, 0).cpu().numpy()
                # Normalize 복구 (대략적)
                debug_img = (debug_img * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])
                debug_img = np.clip(debug_img * 255, 0, 255).astype(np.uint8)
                debug_img = cv2.cvtColor(debug_img, cv2.COLOR_RGB2BGR)
                cv2.imwrite(f'./debug_images/debug_{filename}_{i}.jpg', debug_img)
                saved_debug_count += 1

        if len(frames) == 0:
            results.append({'filename': filename, 'prob': 0.5})
            continue

        # --- 추론 ---
        frames = torch.stack(frames).to(device)
        with torch.no_grad():
            outputs = model(frames)
            probs = torch.sigmoid(outputs)

        avg_prob = probs.mean().item()
        results.append({'filename': filename, 'prob': avg_prob})

    # 결과 저장
    submission = pd.DataFrame(results)
    submission.to_csv(output_path, index=False)

    print(f"✅ Submission Saved: {output_path}")
    print("🔎 예측값 통계 (0.5에 몰려있으면 학습/추론 불일치):")
    print(submission['prob'].describe())

if __name__ == '__main__':
    TEST_DIR = './test_data'
    MODEL_PATH = './model/best_model.pt'
    OUTPUT_PATH = './submission.csv'

    if os.path.exists(TEST_DIR) and os.path.exists(MODEL_PATH):
        inference(TEST_DIR, MODEL_PATH, OUTPUT_PATH)

Overwriting inference.py


In [4]:
!python inference.py

/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(
✅ Inference Start! 파일: 500개
100% 500/500 [10:13<00:00,  1.23s/it]
✅ Submission Saved: ./submission.csv
🔎 예측값 통계 (0.5에 몰려있으면 학습/추론 불일치):
count    5.000000e+02
mean     9.479996e-04
std      7.688760e-03
min      0.000000e+00
25%      5.155419e-18
50%      1.963089e-10
75%      3.654069e-06
max      1.186678e-01
Name: prob, dtype: float64
